# Notes
- actives/embeddings.pkl was generated with batch size 128
- actives/preds.pkl was generated with batch size 1

In [1]:
import pickle
import warnings
from elektronn.utils import generate_grid, model_kwargs, LoadModels
from elektronn.elektronn_ensemble_predict import data_from_rdkit, ElektroNN_Ensemble

with open("ElektroNN/basisfunction_params.pkl", "rb") as file:
    basisfunction_params = pickle.load(file)

model_path = 'ElektroNN/modelparams/04-20-33/'
num_models = 5
map_location = 'cuda'
warnings.filterwarnings("ignore")
models_to_load = [2]
loader_specified = LoadModels(
    models_to_load, num_models, model_path, model_kwargs, map_location, all_models=True
)
loader_specified.load()
models_specified = loader_specified.models

loaded model_fold_1.pth
loaded model_fold_2.pth
loaded model_fold_3.pth
loaded model_fold_4.pth
loaded model_fold_5.pth


In [ ]:
from torch.utils.data import Dataset
from rdkit import Chem
from rdkit.Chem import AllChem

class MolGraphData(Dataset):
    def __init__(self, smiles_dict, basisfunction_params):
        super().__init__()
        self.mols = []
        self.names = []
        self.basisfunction_params = basisfunction_params
        self.params = AllChem.ETKDGv3()
        self.params.randomSeed = 42069
        self.bad_atom_mols = 0
        for name, smiles in smiles_dict.items():
            mol = Chem.MolFromSmiles(smiles)
            bad_atom = False
            for i in range(mol.GetNumAtoms()):
                atomic_number = float(mol.GetAtomWithIdx(i).GetAtomicNum())
                if atomic_number not in self.basisfunction_params:
                    bad_atom = True
                    break
            if bad_atom:
                self.bad_atom_mols += 1
                continue
            self.mols.append( mol )
            self.names.append( name )

    def __len__(self):
        return len(self.mols)
    
    def __getitem__(self, index):
        name = self.names[index]
        mol = self.mols[index]

        mol = Chem.AddHs(mol)
        AllChem.EmbedMolecule(mol, self.params)
        data_mol = data_from_rdkit(mol, name, self.basisfunction_params)

        return data_mol
    
    def get_mol(self, name):
        index = self.names.index(name)
        return self[index]



In [4]:
import csv
import os
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import numpy as np

def save_sdf_files(dataset, sdf_dir):
    """
    Save SDF files for each molecule in the dataset.

    Args:
        dataset (MolGraphData): Dataset containing molecules.
        sdf_dir (str): Directory to save SDF files.
    """
    os.makedirs(sdf_dir, exist_ok=True)
    for name in dataset.names:
        index = dataset.names.index(name)
        mol = dataset.conformers[index]
        sdf_path = os.path.join(sdf_dir, f"{name}.sdf")
        writer = Chem.SDWriter(sdf_path)
        # Write all conformers, setting a unique ID for each
        for cid in range(mol.GetNumConformers()):
            mol.SetProp('ID', f'{name}_conformer_{cid}')
            writer.write(mol, confId=cid)
        writer.close()

# Prepare SMILES dictionary from CSV
smiles_dict = {}

assay_id = 'AID435034'
outcome = 'actives'
filename = f"{assay_id}_{outcome}.csv"
root = './data/qsar'
filepath = os.path.join(root, filename)
sdf_dir = os.path.join(root, 'sdf', assay_id, outcome)
os.makedirs(root, exist_ok=True)

print("Loading SMILES")
with open(filepath) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        smiles = row['SMILES']
        name = row['CID']
        smiles_dict[name] = smiles

print("Creating dataset")
# Create dataset and save SDF files
dataset = MolGraphData(smiles_dict, basisfunction_params)


# Prediction and result saving
dl = DataLoader(dataset, pin_memory=True, batch_size=128, num_workers=8)

max_predictions = 1000
num_predicted = 0

preds = {}
preds_extended = {}

print("Predicting")
for mol in tqdm(dl):
    if num_predicted >= max_predictions:
        break
    mol.to(map_location)
    pred = ElektroNN_Ensemble(mol, map_location, models_specified)
    for i in range(len(mol.filename)):
        if num_predicted >= max_predictions:
            break
        filename = mol.filename[i]
        pred_i = pred[mol.batch == i]
        preds[filename] = pred_i

        # Build extended prediction with coordinates and atomic numbers
        molecule = dataset.get_mol(filename)
        pos = molecule.pos.cpu().numpy()  # shape: (N, 3)
        atom_numbers = molecule.atomic_numbers.cpu().numpy()  # shape: (N,)
        pred_np = pred_i.cpu().numpy()  # shape: (N, 127)
        atom_numbers = atom_numbers.reshape(-1, 1)
        preds_extended[filename] = np.concatenate([pred_np, pos, atom_numbers], axis=1)
        num_predicted += 1

# print("Saving SDF files")
# save_sdf_files(dataset, sdf_dir)

print("Saving predictions")
output_root = os.path.join(root, 'embeddings', assay_id, outcome)
os.makedirs(output_root, exist_ok=True)
import pickle
with open(os.path.join(output_root, "embeddings.pkl"), "wb") as f:
    pickle.dump(preds, f)

with open(os.path.join(output_root, "embeddings_extended.pkl"), "wb") as f:
    pickle.dump(preds_extended, f)


Loading SMILES
Creating dataset
Predicting


  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:04<00:00,  4.03s/it]

Saving predictions


In [53]:
preds_extended["1248737"][1,127:]

array([4.38622379, 0.21188274, 0.15407038, 8.        ])

In [5]:
bs = pickle.load(open('data/qsar/embeddings/AID435034/actives/embeddings.pkl', 'rb'))
pred = pickle.load(open('data/qsar/embeddings/AID435034/actives/preds.pkl', 'rb'))


In [6]:
preds['1263872'] == bs['1263872']

tensor([[True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        ...,
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True],
        [True, True, True,  ..., True, True, True]], device='cuda:0')

In [49]:
%load_ext autoreload
%autoreload 2
from elektronn.utils import coeff_unperm_gau2grid_density_kdtree_ml_only, write_cube_file
import numpy as np

Rs = [(14, 0), (14, 1), (5, 2), (4, 3), (2, 4)]

n=0
mls = {}
preds = pickle.load(open('data/qsar/embeddings/AID435034/actives/preds.pkl', 'rb'))
display(preds)
output_path = f"data/qsar/densities/{assay_id}/actives/"
os.makedirs(output_path, exist_ok=True)
for name, pred in preds.items():
    molecule = dataset.get_mol(name)
    print("Generate Grid for", name)
    x, y, z, vol, x_spacing, y_spacing, z_spacing = generate_grid(molecule, spacing=0.25, buffer=2.0)
    # Ensure x, y, z are float64 for gau2grid
    x_flat = x.flatten().astype(np.float64)
    y_flat = y.flatten().astype(np.float64)
    z_flat = z.flatten().astype(np.float64)
    ml = coeff_unperm_gau2grid_density_kdtree_ml_only(x_flat, y_flat, z_flat, molecule, pred, Rs)
    mls[name] = {"x": x, "y": y, "z": z, "molecule": molecule, "ml": ml}
    n+=1
    write_cube_file(output_path + f"{name}.cube", molecule.atomic_numbers, molecule.pos, x,y,z,ml)   
    if n == 5:
        break

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'1263872': tensor([[-0.0062, -0.0023,  0.0354,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0169,  0.0114,  0.0438,  ...,  0.0000,  0.0000,  0.0000],
         [-0.0038,  0.0118,  0.0325,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0098,  0.0961,  0.4694,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0164,  0.1060,  0.4604,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0109,  0.1023,  0.4495,  ...,  0.0000,  0.0000,  0.0000]],
        device='cuda:0'),
 '655606': tensor([[-3.1931e-04, -5.0482e-03,  3.5540e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 3.6097e-03,  6.5958e-03,  4.5104e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.3938e-03,  1.0736e-02,  2.0417e-02,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [ 1.5252e-02,  1.1241e-01,  4.1912e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [ 1.6798e-02,  9.6724e-02,  4.3298e-01,  ...,  0.0000e+00,
          

Generate Grid for 1263872
Generate Grid for 655606
Generate Grid for 3236651
Generate Grid for 2998899
Generate Grid for 3239606


In [7]:
mls

{'1263872': {'x': array([[[-9.980658, -9.980658, -9.980658, ..., -9.980658, -9.980658,
           -9.980658],
          [-9.980658, -9.980658, -9.980658, ..., -9.980658, -9.980658,
           -9.980658],
          [-9.980658, -9.980658, -9.980658, ..., -9.980658, -9.980658,
           -9.980658],
          ...,
          [-9.980658, -9.980658, -9.980658, ..., -9.980658, -9.980658,
           -9.980658],
          [-9.980658, -9.980658, -9.980658, ..., -9.980658, -9.980658,
           -9.980658],
          [-9.980658, -9.980658, -9.980658, ..., -9.980658, -9.980658,
           -9.980658]],
  
         [[-9.85601 , -9.85601 , -9.85601 , ..., -9.85601 , -9.85601 ,
           -9.85601 ],
          [-9.85601 , -9.85601 , -9.85601 , ..., -9.85601 , -9.85601 ,
           -9.85601 ],
          [-9.85601 , -9.85601 , -9.85601 , ..., -9.85601 , -9.85601 ,
           -9.85601 ],
          ...,
          [-9.85601 , -9.85601 , -9.85601 , ..., -9.85601 , -9.85601 ,
           -9.85601 ],
          

In [13]:
from elektronn.utils import write_cube_file
Rs = [(14,0),(14,1),(5,2),(4,3),(2,4)]



print("Generated Grid")
print("Generate ML object")

print("Generated ML Object")
print("Generate Cube")
output_path = f"data/qsar/densities/{assay_id}/{outcome}/"
os.makedirs(output_path, exist_ok=True)
for name, ml in mls.items():
    write_cube_file(output_path + f"{name}.cube", ml["molecule"].atomic_numbers, ml["molecule"].pos, ml["x"], ml["y"], ml["z"], ml["ml"])       

print("Generated Cube")

Generated Grid
Generate ML object
Generated ML Object
Generate Cube
Generated Cube


In [ ]:
# save mls as pickle 
import pickle
with open("mls.pkl", "wb") as file:
    pickle.dump(mls, file)


In [ ]:
import nglview as nv
import pickle
os 
# Create a new NGLView instance
view = nv.NGLWidget()

for density in os.listdir('data/qsar/densities/'):
    view.add_component(os.path.join('data/qsar/densities/', density))
    sdf = 

# Display the view
display(view)

FileNotFoundError: [Errno 2] No such file or directory: 'mls.pkl'